In [ ]:
import numpy as np
import pandas as pd
from glob import glob
from ALLCools.mcds import RegionDS
from scipy.sparse import csr_matrix, save_npz


In [ ]:
group_name = 'AMY'


In [ ]:
region = group_name

In [ ]:
region_ds = RegionDS.open(sorted(glob(f'/rs2/221126-dmr/*/*/{region}/{region}-chr*.dmr.ds')), region_dim='dmr')
region_ds['frac'] = region_ds['data'].sel(count_type='mc') / region_ds['data'].sel(count_type='cov')
data = region_ds['frac'].to_pandas()
cov = region_ds['data'].sel({'count_type':'cov'}).to_pandas()


In [ ]:
hypo_frac, median_frac, hyper_frac = np.nanquantile(data[cov>5], (0.2, 0.5, 0.8), axis=0)
tmp = median_frac - 0.3
col_hypo = np.where((data<np.min([hypo_frac, np.where(tmp<0.2, 0.2, tmp)], axis=0)) & (cov>5))
tmp = median_frac + 0.2
col_hyper = np.where((data>np.max([hyper_frac, np.where(tmp>0.8, 0.8, tmp)], axis=0)) & (cov>5))
# col_hypo = np.where((data<hypo_frac) & (cov>5))
# col_hyper = np.where((data>hyper_frac) & (cov>5))
col_hypo = csr_matrix((np.ones(len(col_hypo[0])), (col_hypo[0], col_hypo[1])), data.shape)
col_hyper = csr_matrix((np.ones(len(col_hyper[0])), (col_hyper[0], col_hyper[1])), data.shape)


In [ ]:
hypo_frac, hyper_frac = np.nanquantile(data[cov>5], (0.2, 0.8), axis=1)
row_hypo = np.where((data<hypo_frac[:,None]) & (cov>5))
row_hyper = np.where((data>hyper_frac[:,None]) & (cov>5))
row_hypo = csr_matrix((np.ones(len(row_hypo[0])), (row_hypo[0], row_hypo[1])), data.shape)
row_hyper = csr_matrix((np.ones(len(row_hyper[0])), (row_hyper[0], row_hyper[1])), data.shape)


In [ ]:
total_hypo = row_hypo.multiply(col_hypo)
total_hyper = row_hyper.multiply(col_hyper)
save_npz(f'{region}_hypo.npz', total_hypo)
save_npz(f'{region}_hyper.npz', total_hyper)


In [ ]:
print(pd.Series(total_hypo.sum(axis=1).A1).describe())


In [ ]:
print(pd.Series(total_hyper.sum(axis=1).A1).describe())
